In [2]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
import ipywidgets as widgets
from IPython.display import display, HTML, Image
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Define the Michaelis-Menten function
def michaelis_menten(S, Vmax, Km):
    return Vmax * S / (Km + S)

# Define instruction HTML for each experiment
instructions = {
    'MM-Kinetics': """
    <h1>Analyzing Alkaline Phosphatase Enzyme Kinetics Using the Michaelis-Menten Model</h1>
    <h2>Step 1: Import Your Data</h2>
    <p>Please copy the pNPP (substrate) concentration and Rate data (columns A and B) from the "M-M Fit" sheet within the "Enzyme kinetics_MM fit" Excel file. Before pasting the data into the input field below, please update the column headers to accurately represent the correct substrate name and concentrations used in your experiment.</p>
    <img src="..." alt="Screenshot of example data">
    <p>The data pasted from Excel should look something like the following:</p>
    <img src="..."alt="Screenshot of example data">
    <p><strong>Check that the values and units match those from your experiment before proceeding.</strong></p>
    <h2>Step 2: Generate the Kinetics Curve</h2>
    <p>Click the "Plot Data" button to visualize your results and apply the Michaelis-Menten curve fitting.</p>
    <p><em><strong>Remember to download the resulting graph and incorporate it into your Excel file.</strong></em></p>
    <p>You'll need both the visual representation and the raw data in Excel for your complete report.</p>
    """,
    'MM-Inhibition': """
    <h1>Michaelis-Menten Inhibition</h1>
    <h2>Analyzing Alkaline Phosphatase Enzyme Kinetics Using the Michaelis-Menten Model</h2>
    <h2>Step 1: Import Your Data</h2>
    <p>Copy the inhibitor concentration and percent inhibition data from columns A and B on the "Raw Data" sheet of your "Enzyme kinetics_MM fit_Inhibition" Excel file. Before pasting the data into the input field below, please update the column headers to accurately represent the specific inhibitors and concentrations used in your experiment.</p>
    <img src="..." alt="Screenshot of example data">
    <p>The data pasted from Excel should look something like the following:</p>
    <img src="data:image/png;base64,
..."alt="Screenshot of example data">
    <p><strong>Check that the values and units match those from your experiment before proceeding.</strong></p>
    <p>If you do not see the Data Input box, click on the Runtime tab and select "Run All".</p>
    <h2>Step 2: Generate the Kinetics Curve</h2>
    <p>Click the "Plot Data" button to visualize your results and apply the curve fitting.</p>
    <p><em><strong>Remember to download the resulting graph and incorporate it into your Excel file.</strong></em></p>
    <p>You'll need both the visual representation and the raw data in Excel for your complete report.</p>
    """,
    'MM-Inhibition-Week2': """
    <h1>Michaelis-Menten Inhibition Week Two </h1>
    <h2>Analyzing Alkaline Phosphatase Enzyme Kinetics Using the Michaelis-Menten Model</h2>
    <h2>Step 1: Import Your Data</h2>
    <p>Copy the inhibitor concentration and Velocity data from columns A and B on the \"Raw Data\" sheet of your \"Enzyme kinetics_MM fit_Inhibition\" Excel file. Before pasting the data into the input field below, please update the column headers to accurately represent the specific inhibitors and concentrations used in your experiment.</p>
    <img src="..." alt="Screenshot of example data">
    <p>The data pasted from Excel should look something like the following:</p>
    <img src="..."alt="Screenshot of example data">
    <p><strong>Check that the values and units match those from your experiment before proceeding.</strong></p>
    <p>If you do not see the Data Input box, click on the Runtime tab and select "Run All".</p>
    <h2>Step 2: Generate the Kinetics Curve</h2>
    <p>Click the "Plot Data" button to visualize your results and apply the curve fitting..</p>
    <p><em><strong>Remember to download the resulting graph and incorporate it into your Excel file.</strong></em></p>
    <p>You'll need both the visual representation and the raw data in Excel for your complete report.</p>
    """
}

# Create widgets
experiment_dropdown = widgets.Dropdown(
    options=['MM-Kinetics', 'MM-Inhibition', 'MM-Inhibition-Week2'],
    value='MM-Kinetics',
    description='Experiment:',
)

instructions_output = widgets.Output()
experiment_output = widgets.Output()

# Function to update instructions
def update_instructions(change):
    with instructions_output:
        instructions_output.clear_output()
        display(HTML(instructions[change['new']]))

# Function to display experiment UI
def display_experiment_ui(change):
    with experiment_output:
        experiment_output.clear_output(wait=True)
        if change['new'] == 'MM-Kinetics':
            display_kinetics_ui()
        elif change['new'] == 'MM-Inhibition':
            display_inhibition_ui()
        elif change['new'] == 'MM-Inhibition-Week2':
            display_inhibition_week2_ui()

def display_kinetics_ui():
    data_textarea = widgets.Textarea(
        placeholder='Paste your data here from Excel',
        description='Data Input:',
        layout={'width': '100%', 'height': '200px'}
    )
    display(data_textarea)

    process_data_button = widgets.Button(description="Plot Data")
    display(process_data_button)

    def process_data(b):
        try:
            # Convert the pasted data into a DataFrame
            data_io = io.StringIO(data_textarea.value)
            df = pd.read_csv(data_io, sep="\t", header=0)
            # Sort data by substrate concentration just in case
            df.sort_values(by=df.columns[0], inplace=True)
            # Initial guess for parameters Vmax and Km
            initial_guess = [max(df[df.columns[1]]), np.median(df[df.columns[0]])]
            # Perform curve fitting
            popt, _ = curve_fit(michaelis_menten, df[df.columns[0]], df[df.columns[1]], p0=initial_guess)
            # Calculate the R² value
            predicted_values = michaelis_menten(df[df.columns[0]], *popt)
            r2 = r2_score(df[df.columns[1]], predicted_values)
            # Plot the data and the curve
            plt.figure(figsize=[8,6])
            plt.scatter(df[df.columns[0]], df[df.columns[1]], label='Experimental Data')
            plt.plot(np.linspace(0, max(df[df.columns[0]]), 500),
                     michaelis_menten(np.linspace(0, max(df[df.columns[0]]), 500), *popt),
                     'r-', label=f'MM Fit: Vmax={popt[0]:.4f}, Km={popt[1]:.4f}, R²={r2:.4f}')
            plt.title('Alkaline Phosphatase Michaelis-Menten Kinetics')
            plt.xlabel(df.columns[0])
            plt.ylabel(df.columns[1])
            plt.legend()
            plt.grid(False)
            plt.show()
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check your input data and try again.")

    process_data_button.on_click(process_data)

def display_inhibition_ui():
    data_textarea = widgets.Textarea(
        placeholder='Paste your data here from Excel',
        description='Data Input:',
        layout={'width': '100%', 'height': '200px'}
    )
    process_data_button = widgets.Button(description="Plot Data")

    # Create a VBox to hold both the textarea and the button
    vbox = widgets.VBox([data_textarea, process_data_button])
    display(vbox)

    def process_data(b):
        try:
            raw_data = data_textarea.value
            data_io = io.StringIO(raw_data)
            df = pd.read_csv(data_io, sep="\t", header=0)

            df.sort_values(by=df.columns[0], inplace=True)

            initial_guess = [max(df[df.columns[2]]), np.median(df[df.columns[0]])]

            popt, _ = curve_fit(michaelis_menten, df[df.columns[0]], df[df.columns[2]], p0=initial_guess)

            predicted_values = michaelis_menten(df[df.columns[0]], *popt)
            r2 = r2_score(df[df.columns[2]], predicted_values)

            plt.figure(figsize=[8,6])
            plt.scatter(df[df.columns[0]], df[df.columns[2]], label='Experimental Data')
            plt.plot(np.linspace(0, max(df[df.columns[0]]), 500),
                     michaelis_menten(np.linspace(0, max(df[df.columns[0]]), 500), *popt),
                     'r-', label=f'Fit: Imax={popt[0]:.4f}, IC50={popt[1]:.4f}, R²={r2:.4f}')
            plt.title('Alkaline Phosphatase Inhibition')
            plt.xlabel(df.columns[0])
            plt.ylabel(df.columns[2])
            plt.legend()
            plt.grid(False)
            plt.show()
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check your input data and try again.")

    process_data_button.on_click(process_data)

def display_inhibition_week2_ui():
    # Create a VBox to hold all the widgets
    vbox = widgets.VBox()

    # Create Textarea widgets for each dataset
    data_textareas = [widgets.Textarea(
        placeholder=f'Paste your data here for α{i+1} dataset from Excel',
        description=f'α{i+1}:',
        layout={'width': '100%', 'height': '150px'}
    ) for i in range(4)]

    # Create a button that will process the pasted data
    process_data_button = widgets.Button(description="Plot Data")

    # Add all widgets to the VBox
    vbox.children = data_textareas + [process_data_button]

    # Display the VBox
    display(vbox)

    # Function to process and plot a dataset
    def process_and_plot_dataset(df, ax, label_prefix):
        # Sort data by Substrate Concentration just in case
        df.sort_values(by=df.columns[0], inplace=True)

        # Initial guess for parameters Vmax and Km
        initial_guess = [max(df[df.columns[1]]), np.median(df[df.columns[0]])]

        # Perform curve fitting
        popt, _ = curve_fit(michaelis_menten, df[df.columns[0]], df[df.columns[1]], p0=initial_guess)

        # Calculate the R² value
        predicted_values = michaelis_menten(df[df.columns[0]], *popt)
        r2 = r2_score(df[df.columns[1]], predicted_values)

        # Plot the data and the fitted curve
        ax.scatter(df[df.columns[0]], df[df.columns[1]], label=f'{label_prefix} Experimental Data')
        ax.plot(np.linspace(0, max(df[df.columns[0]]), 500),
                michaelis_menten(np.linspace(0, max(df[df.columns[0]]), 500), *popt),
                label=f'{label_prefix} Fit: Vmax={popt[0]:.4f}, Km={popt[1]:.4f}, R²={r2:.4f}')

    def process_data(b):
        try:
            fig, ax = plt.subplots(figsize=[10,8])

            for i, textarea in enumerate(data_textareas):
                # Convert the pasted data into a DataFrame
                raw_data = textarea.value
                if raw_data.strip():  # Only process non-empty textareas
                    data_io = io.StringIO(raw_data)
                    df = pd.read_csv(data_io, sep="\t", header=0)
                    process_and_plot_dataset(df, ax, f'α{i+1}')

            plt.title('Alkaline Phosphatase Kinetics with Different Inhibitor Amounts')
            plt.xlabel(df.columns[0])
            plt.ylabel(df.columns[1])
            plt.legend()
            plt.grid(False)
            plt.show()
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check your input data and try again.")

    process_data_button.on_click(process_data)

# Observe changes in dropdown
experiment_dropdown.observe(update_instructions, names='value')
experiment_dropdown.observe(display_experiment_ui, names='value')

# Display widgets
display(HTML("<h1>Analyzing Alkaline Phosphatase Enzyme Kinetics Using the Michaelis-Menten Model</h1>"))
display(experiment_dropdown)
display(instructions_output)
display(experiment_output)

# Initial displays
update_instructions({'new': experiment_dropdown.value})
display_experiment_ui({'new': experiment_dropdown.value})

Dropdown(description='Experiment:', options=('MM-Kinetics', 'MM-Inhibition', 'MM-Inhibition-Week2'), value='MM…

Output()

Output()